In [ ]:
import pandas as pd
import ast
import requests
from bs4 import BeautifulSoup
# from IPython.core.display import display, HTML
pd.set_option("display.max_colwidth", None)

/var/folders/_t/nz76vbhx0xdgpnky43xkhs440000gn/T/ipykernel_69751/446727533.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
TARGET_PATH = "./../data/archived_snapshots_urls.csv"
recorded_urls = pd.read_csv(TARGET_PATH)
recorded_urls = recorded_urls[recorded_urls.archived_snapshots != '{}'].reset_index(drop=True)
recorded_urls['archived_snapshots'] = recorded_urls.archived_snapshots.apply(ast.literal_eval)
archived_snapshots_expanded = pd.json_normalize(recorded_urls.archived_snapshots)
recorded_urls = recorded_urls.drop(columns=["archived_snapshots"]).join(archived_snapshots_expanded)

In [3]:
recorded_urls

,url,timestamp,closest.status,closest.available,closest.url,closest.timestamp
0,mastodon.xyz,20220101000000,200,True,http://web.archive.org/web/20221122080738/https://mastodon.xyz/,20221122080738
1,mastodon.xyz,20220401000000,200,True,http://web.archive.org/web/20221122080738/https://mastodon.xyz/,20221122080738
2,mastodon.xyz,20220701000000,200,True,http://web.archive.org/web/20221122080738/https://mastodon.xyz/,20221122080738
3,mastodon.xyz,20221001000000,200,True,http://web.archive.org/web/20221122080738/https://mastodon.xyz/,20221122080738
4,mastodon.xyz,20230101000000,200,True,http://web.archive.org/web/20230101003413/https://mastodon.xyz/,20230101003413
...,...,...,...,...,...,...
3080,osna.social,20241001000000,200,True,http://web.archive.org/web/20241008045235/https://osna.social/,20241008045235
3081,osna.social,20250101000000,200,True,http://web.archive.org/web/20250116195618/https://osna.social/,20250116195618
3082,osna.social,20250401000000,200,True,http://web.archive.org/web/20250405201845/https://osna.social/,20250405201845
3083,osna.social,20250701000000,200,True,http://web.archive.org/web/20250622043007/https://osna.social/,20250622043007


In [4]:
recorded_urls['closest.url']

0       http://web.archive.org/web/20221122080738/https://mastodon.xyz/
1       http://web.archive.org/web/20221122080738/https://mastodon.xyz/
2       http://web.archive.org/web/20221122080738/https://mastodon.xyz/
3       http://web.archive.org/web/20221122080738/https://mastodon.xyz/
4       http://web.archive.org/web/20230101003413/https://mastodon.xyz/
                                     ...                               
3080     http://web.archive.org/web/20241008045235/https://osna.social/
3081     http://web.archive.org/web/20250116195618/https://osna.social/
3082     http://web.archive.org/web/20250405201845/https://osna.social/
3083     http://web.archive.org/web/20250622043007/https://osna.social/
3084     http://web.archive.org/web/20250930221414/https://osna.social/
Name: closest.url, Length: 3085, dtype: object

In [10]:
import requests

url = "https://web.archive.org/web/20221217083518/https://mastodon.xyz/about"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "de-DE,de;q=0.9,en;q=0.8",
    "Cache-Control": "no-cache",
    "Pragma": "no-cache",
}
response = requests.get(url, headers=headers, timeout=30, allow_redirects=True)
response.raise_for_status()

open("page.html", "w", encoding="utf-8").write(response.text)

32952

In [6]:
response.__dict__

{'_content': b'<!DOCTYPE html>\n<html lang="en">\n<head><script src="//archive.org/includes/athena.js" type="text/javascript"></script>\n<script type="text/javascript">window.addEventListener(\'DOMContentLoaded\',function(){var v=archive_analytics.values;v.service=\'wb\';v.server_name=\'wwwb-app223.us.archive.org\';v.server_ms=286;archive_analytics.send_pageview({});});</script>\n<script type="text/javascript" src="https://web-static.archive.org/_static/js/bundle-playback.js?v=Hk4-lkW3" charset="utf-8"></script>\n<script type="text/javascript" src="https://web-static.archive.org/_static/js/wombat.js?v=txqj7nKC" charset="utf-8"></script>\n<script>window.RufflePlayer=window.RufflePlayer||{};window.RufflePlayer.config={"autoplay":"on","unmuteOverlay":"hidden","showSwfDownload":true};</script>\n<script type="text/javascript" src="https://web-static.archive.org/_static/js/ruffle/ruffle.js"></script>\n<script type="text/javascript">\n    __wm.init("https://web.archive.org/web");\n  __wm.womb

In [11]:
from bs4 import BeautifulSoup
import re

html = response.text.lower()
soup = BeautifulSoup(html, "html.parser")

out = {}

# "About" (best available in this HTML): meta description + title
title = soup.title.get_text(strip=True) if soup.title else None
meta_desc = soup.select_one('meta[name="description"]')
about = meta_desc["content"].strip() if meta_desc and meta_desc.has_attr("content") else None

out["About"] = {
    "title": title,
    "description": about,
}

# Helper: try to find a section by heading text (works only if content exists in HTML)
def extract_section_by_heading(heading_regex):
    heading = soup.find(
        lambda t: t.name in ["h1","h2","h3","h4","h5","h6"]
        and t.get_text(" ", strip=True)
        and re.search(heading_regex, t.get_text(" ", strip=True), re.I)
    )
    if not heading:
        return None

    # Collect text until the next heading
    parts = []
    for sib in heading.find_all_next():
        if sib.name in ["h1","h2","h3","h4","h5","h6"] and sib is not heading:
            break
        if sib.name in ["p","li","div","section"]:
            txt = sib.get_text(" ", strip=True)
            if txt:
                parts.append(txt)

    return "\n".join(parts).strip() or None

# Try to extract (will return None for your HTML)
out["Server Rules"] = extract_section_by_heading(r"(server\s*rules|regeln|rules)")
out["Moderated Servers"] = extract_section_by_heading(r"(moderated\s*servers|moderated|domain\s*blocks|blocklist|server\s*block)")

# Show whether the HTML even contains those phrases anywhere
full_text = soup.get_text(" ", strip=True).lower()
out["_contains_keywords"] = {
    "rules_word_present": "rules" in full_text or "regeln" in full_text,
    "moderated_word_present": "moderated" in full_text,
    "domain_blocks_word_present": "domain_blocks" in full_text,
}

print(out)


{'About': {'title': 'about - mastodon', 'description': 'the social network of the future: no ads, no corporate surveillance, ethical design, and decentralization! own your data with mastodon!'}, 'Server Rules': None, 'Moderated Servers': None, '_contains_keywords': {'rules_word_present': False, 'moderated_word_present': False, 'domain_blocks_word_present': False}}


In [12]:
out

{'About': {'title': 'about - mastodon',
  'description': 'the social network of the future: no ads, no corporate surveillance, ethical design, and decentralization! own your data with mastodon!'},
 'Server Rules': None,
 'Moderated Servers': None,
 '_contains_keywords': {'rules_word_present': False,
  'moderated_word_present': False,
  'domain_blocks_word_present': False}}